# Music Streaming App - Data Modeling With Apache Cassandra

**Part I. ETL Pipeline for Pre-Processing the Files**

#### Import Python packages 

In [63]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [64]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/vaibhavbansal/data_engineering/Music-App-Data-Modeling-with-Apache-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [65]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)    
        for line in csvreader:
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [66]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


**Part II. Apache Cassandra Project.**

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [67]:
import cassandra
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [68]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_streaming_app 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [69]:
try:
    session.set_keyspace('music_streaming_app')
except Exception as e:
    print(e)

### With Apache Cassandra we model the database tables on the queries you want to run.

## Creating tables based on these given queries

**1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**


**2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**
    

**3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**




## Query 1
Description:**Have taken sessionId as Partition Key and itemInSession as clustering column to make primary key unique**

In [70]:
try:
    session.execute('CREATE TABLE IF NOT EXISTS music_library (artist text,song_title text,song_length float,sessionId int,itemInSession int, PRIMARY KEY(sessionId,itemInSession))')
except Exception as e:
    print(e)
                    

**Have taken sessionId as Partition Key and itemInSession as clustering column to make primary key unique**

In [71]:
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [72]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_library (artist,song_title,song_length,sessionId,itemInSession)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9],float(line[5]),int(line[8]),int(line[3])))

**Verifying that the data have been inserted into each table**

In [73]:
try:
    rows = session.execute('SELECT * FROM music_library WHERE sessionId=338 and itemInSession = 4')
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song_title,row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2 
Description : **Have taken userId and sessionId combined as Partition Key to make primary key unique and itemInSession as clustering column for sorting**

In [74]:
try:
    session.execute('CREATE TABLE IF NOT EXISTS user_stream_info_library (userId int,sessionId int,artist text,song_title text,firstName text,lastName text,itemInSession int, PRIMARY KEY((userId,sessionId),itemInSession))')
except Exception as e:
    print(e)
                                  

**Have taken userId and sessionId combined as Partition Key to make primary key unique and itemInSession as clustering column for sorting**

In [75]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_stream_info_library (userId,sessionId,artist,song_title,firstName,lastName,itemInSession)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),line[0],line[9],line[1],line[4],int(line[3])))

In [76]:
try:
    rows = session.execute("SELECT artist,song_title,firstName,lastName FROM user_stream_info_library WHERE userId=10 and sessionId = 182")
except Exception as e:
    print(e)
for row in rows:
    print(row.artist,row.song_title,row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3
Description: **Have taken song_title as Partition Key to make primary key unique and userId as clustering column**

In [77]:
try:
    session.execute('CREATE TABLE IF NOT EXISTS user_info (song_title text, userId int,firstName text,lastName text, PRIMARY KEY(song_title,userId))')
except Exception as e:
    print(e)

**Have taken song_title as Partition Key to make primary key unique and userId as clustering column**

In [78]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_info (song_title,userId,firstName,lastName)"
        query = query + " VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [79]:
try:
    rows = session.execute("SELECT  firstName, lastName FROM user_info WHERE song_title = 'All Hands Against His Own'")
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [80]:
try:
    rows = session.execute('DROP TABLE music_library;')
except Exception as e:
    print(e)
try:
    rows = session.execute('DROP TABLE user_stream_info_library;')
except Exception as e:
    print(e)
try:
    rows = session.execute('DROP TABLE user_info;')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [81]:
session.shutdown()
cluster.shutdown()